In [1]:
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers==0.0.25
!pip install .[bitsandbytes]

Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 10620, done.
remote: Counting objects: 100% (1437/1437), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 10620 (delta 1225), reused 1288 (delta 1140), pack-reused 9183
Receiving objects: 100% (10620/10620), 214.39 MiB | 36.02 MiB/s, done.
Resolving deltas: 100% (7824/7824), done.
Updating files: 100% (206/206), done.
/notebooks/LLaMA-Factory
CITATION.cff  README.md     docker-compose.yml  requirements.txt  tests/
Dockerfile    README_zh.md  evaluation/         scripts/
LICENSE       assets/       examples/           setup.py
Makefile      data/         pyproject.toml      src/
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-c95yyr83/unsloth_77bd7beb557d4b66aecc597445ac3865
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-c95yyr83/unsloth_77bd7beb557d4b66aecc597445ac3865
  Resolved https://github.com/unslothai/unsloth.

In [2]:
!nvidia-smi

Fri May  3 12:24:04 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.116.04   Driver Version: 525.116.04   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   40C    P8    16W / 140W |      1MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
try:
    assert torch.cuda.is_available() is True
except AssertionError:
    print("Set up a GPU before using llama factory!")

In [4]:
import json

%cd /notebooks/LLaMA-Factory

MODEL_NAME = "Llama-3"
AUTHOR = "LLaMA Factory"

with open("/notebooks/LLaMA-Factory/data/identity.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

for sample in dataset:
    sample["output"] = sample["output"].replace("MODEL_NAME", MODEL_NAME).replace("AUTHOR", AUTHOR)

with open("/notebooks/LLaMA-Factory/data/identity.json", "w", encoding="utf-8") as f:
    json.dump(dataset, f, indent=2, ensure_ascii=False)

/notebooks/LLaMA-Factory


In [8]:
#%cd /notebooks/LLaMA-Factory
!GRADIO_SHARE=True llamafactory-cli webui

Running on local URL:  http://0.0.0.0:7860
Running on public URL: https://710273a92ee7006dcc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
^C
Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://710273a92ee7006dcc.gradio.live


In [6]:
!pip install 'urllib3<2'

  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 4.28.3 requires urllib3~=2.0, but you have urllib3 1.26.18 which is incompatible.


In [11]:
!pip3 uninstall wandb

Found existing installation: wandb 0.13.4
Uninstalling wandb-0.13.4:
  Would remove:
    /usr/local/bin/wandb
    /usr/local/bin/wb
    /usr/local/lib/python3.9/dist-packages/wandb-0.13.4.dist-info/*
    /usr/local/lib/python3.9/dist-packages/wandb/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [5]:
import json

args = dict(
  stage="sft",                        # do supervised fine-tuning
  do_train=True,
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  dataset="identity,alpaca_gpt4_en",             # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                   # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir="llama3_lora",                  # the path to save LoRA adapters
  per_device_train_batch_size=2,               # the batch size
  gradient_accumulation_steps=4,               # the gradient accumulation steps
  lr_scheduler_type="cosine",                 # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=1000,                      # save checkpoint every 1000 steps
  learning_rate=5e-5,                     # the learning rate
  num_train_epochs=3.0,                    # the epochs of training
  max_samples=500,                      # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                     # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                   # use LoRA+ with lambda=16.0
  use_unsloth=True,                      # use UnslothAI's LoRA optimization for 2x faster training
  fp16=True,                         # use float16 mixed precision training
)

json.dump(args, open("train_llama3.json", "w", encoding="utf-8"), indent=2)

# %cd /notebooks/LLaMA-Factory
# !llamafactory-cli train train_llama3.json

In [7]:
# @title
import os
for k, v in os.environ.items():
  print(k, v)

KUBERNETES_SERVICE_PORT 443
KUBERNETES_PORT tcp://10.43.0.1:443
HOSTNAME n608ut6wud
JUPYTER_TOKEN 33dc278f6cd097624425683e09e65cbf
LD_LIBRARY_PATH /usr/local/cuda-11.6/lib64
HOME /root
PAPERSPACE_NOTEBOOK_REPO_ID rw1hgwf73xmkv7o
PAPERSPACE_CLUSTER_ID clg07azjl
PIP_DISABLE_PIP_VERSION_CHECK 1
KUBERNETES_PORT_443_TCP_ADDR 10.43.0.1
PATH /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/root/.local/bin:/usr/local/cuda-11.6/bin
KUBERNETES_PORT_443_TCP_PORT 443
DISABLE_JUPYTER true
KUBERNETES_PORT_443_TCP_PROTO tcp
PAPERSPACE_METRIC_WORKLOAD_TYPE notebook
LANG C.UTF-8
DEBIAN_FRONTEND noninteractive
SHELL /bin/bash
APT_INSTALL apt-get install -y --no-install-recommends
KUBERNETES_SERVICE_PORT_HTTPS 443
KUBERNETES_PORT_443_TCP tcp://10.43.0.1:443
PAPERSPACE_FQDN n608ut6wud.clg07azjl.paperspacegradient.com
KUBERNETES_SERVICE_HOST 10.43.0.1
PWD /notebooks
GIT_CLONE git clone --depth 10
PAPERSPACE_NOTEBOOK_ID n608ut6wud
NVIDIA_VISIBLE_DEVICES GPU-8bf12285-6464-3924-0b57-4e476e81d88a


In [9]:
import json

args = dict(
  model_name_or_path="unsloth/llama-3-8b-Instruct-bnb-4bit", # use bnb-4bit-quantized Llama-3-8B-Instruct model
  #adapter_name_or_path="llama3_lora",            # load the saved LoRA adapters
  finetuning_type="lora",                  # same to the one in training
  template="llama3",                     # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
  use_unsloth=True,                     # use UnslothAI's LoRA optimization for 2x faster generation
)

json.dump(args, open("infer_llama3.json", "w", encoding="utf-8"), indent=2)

# %cd /notebooks/LLaMA-Factory
# !llamafactory-cli chat infer_llama3.json